# Articles tables
This notebook contains the code required to reproduce the table shown in the pseudo batch transformation article. 

## loading fedbatch data

In [25]:
import pathlib
import pandas as pd
from patsy import dmatrices
import statsmodels.api as sm
import numpy as np

from pseudobatch import pseudobatch_transform_pandas, preprocess_gaseous_species
from pseudobatch.datasets._dataloaders import _prepare_simulated_dataset

In [26]:
# import the dataset from the article/data folder, this makes sure if the simulations are rerun the new data is used
data_path = pathlib.Path('../data/standard_fed-batch_process.csv')
fedbatch_df = _prepare_simulated_dataset(data_path)

Make a dataframe that only contains the measurements at the sampling time points.

In [27]:
fedbatch_df_measurements_only = (fedbatch_df
    .query('sample_volume > 0')
    .copy()
    .reset_index(drop=True)
)

We will now do the pseudo batch transformation of the simulated measurements.

In [28]:
glucose_in_feed = fedbatch_df_measurements_only['s_f'].iloc[0]

fedbatch_df_measurements_only[["c_Biomass_pseudo", "c_Glucose_pseudo", "c_Product_pseudo"]] = pseudobatch_transform_pandas(
    fedbatch_df_measurements_only,
    measured_concentration_colnames=["c_Biomass", "c_Glucose", "c_Product"],
    reactor_volume_colname="v_Volume",
    accumulated_feed_colname="v_Feed_accum",
    sample_volume_colname="sample_volume",
    concentration_in_feed=[0, glucose_in_feed, 0],
)

## Calculate the growth rate using the pseudo batch transformation

Now we can calculate the corrected biomass using the pseudo batch transformation

In [29]:
def fit_ols_model(formula_like: str, data: pd.DataFrame) -> sm.regression.linear_model.RegressionResultsWrapper:
    y, X = dmatrices(formula_like, data)
    model = sm.OLS(endog=y, exog=X)
    res = model.fit()
    return res

Now we can fit the growth rate for both the transform and raw biomass data.

In [30]:
res_mu_hat_corrected = fit_ols_model("np.log(c_Biomass_pseudo) ~ timestamp", fedbatch_df_measurements_only)
res_mu_hat_noncorrected = fit_ols_model("np.log(m_Biomass) ~ timestamp", fedbatch_df_measurements_only)

## Calculate yields using the corrected fedbatch data
....

We will first estimate the biomass based substrate yield. To estimate this for the raw data we first need to calculate the consumed glucose time series. This is simply done using the following equation:

$$
cm_{Glucose}(t) = m_{Glucose}(0) + \int_0^t v_{feed}(t) dt \cdot c_{Glucose \: feed} - m_{Glucose}(t)
$$

where $cm_{Glucose}(t)$ is the consumed mass of glucose at timepoint $t$, $m_{Glucose}(t)$ is the mass of glucose at timepoint $t$, $v_{feed}(t)$ is the feeding profile (the time integral is the accumulated feed), $c_{Glucose \: feed}$ is the concentration of glucose in the feeding medium and $m_{Glucose}(t)$ is the measured mass of glucose at time $t$.

In [31]:
initial_glucose = fedbatch_df_measurements_only['m_Glucose'].iloc[0]
fedbatch_df_measurements_only['m_Glucose_consumed'] = fedbatch_df_measurements_only['v_Feed_accum'] * glucose_in_feed - fedbatch_df_measurements_only['m_Glucose'] - initial_glucose

The pseudobatch transformation automatically integrates the feeding volume, thus for the pseudo concentrations we don't need to do further processing. Now we are ready to estimate the yield coefficients. We will follow the standard of yield coefficient being positive.

In [32]:
res_yxs_noncorrected = fit_ols_model(formula_like = "m_Glucose_consumed ~ m_Biomass", data= fedbatch_df_measurements_only)
res_yxs_corrected = fit_ols_model(formula_like = "c_Glucose_pseudo ~ c_Biomass_pseudo", data= fedbatch_df_measurements_only)

print(f"Fitted Yxs from raw data: {np.abs(res_yxs_noncorrected.params[1]).round(5)}")
print(f"Fitted Yxs from pseudo batch transformed data: {np.abs(res_yxs_corrected.params[1]).round(5)}")
print(f"True Yxs: {fedbatch_df.Yxs.iloc[0].round(5)}")

Fitted Yxs from raw data: 3.35819
Fitted Yxs from pseudo batch transformed data: 1.85
True Yxs: 1.85


We see that substrate yield coefficient is correctly calculated using the pseudo concentrations, while the estimate is wrong using the non-transformed data.

## Calculate product yield
We will now proceed to the biomass based product yield. The simulated fermentation process has to products: a generic product and CO2.

In [33]:
res_yxp_noncorrected = fit_ols_model(formula_like = "m_Product ~ m_Biomass", data= fedbatch_df_measurements_only)
res_yxp_corrected = fit_ols_model(formula_like = "c_Product_pseudo ~ c_Biomass_pseudo", data= fedbatch_df_measurements_only)

print(f"Fitted Yxp from raw data: {res_yxp_noncorrected.params[1].round(5)}")
print(f"Fitted Yxp from pseudo batch transformed data: {res_yxp_corrected.params[1].round(5)}")
print(f"True Yxp: {fedbatch_df.Yxp.iloc[0].round(5)}")

Fitted Yxp from raw data: 0.83926
Fitted Yxp from pseudo batch transformed data: 0.82151
True Yxp: 0.82151


For the product yield coefficient the error of not using the pseudo batch transformation is small, but still the yield based on the pseudo concentrations are more accurate.

## Calculate CO2 yield
Most of the CO2 evaporates from the bioreactor, thus is is not removed through sample withdrawal. However as we will see we do not obtain a good estimate of the yield coefficient using the raw data here either. 

Measurements of CO2 are usually obtained through an off-gas analyzer that analyze the amount of CO2 passing through. To obtain the correct estimate, we need to preprocess these measurements in to a hypothetical liquid concentration as if the CO2 molecules did not evaporate. We can do this using the function `preprocess_gaseous_species()` and then pseudo batch transform the preprocessed data.

The preprocessing assumes that the amount of CO2 in the liquid is neglectable compared to the accumulated product of CO2. We believe that this is often true in real fermentation processes. For testing purposes and simplification, the simulated fedbatch process assumes that all CO2 evaporates instantly after production.

In [34]:
fedbatch_df_measurements_only['preprocessed_CO2'] = preprocess_gaseous_species(
    accumulated_amount_of_gaseous_species=fedbatch_df_measurements_only['m_CO2_gas'].values,
    reactor_volume=fedbatch_df_measurements_only['v_Volume'],
    sample_volume=fedbatch_df_measurements_only['sample_volume']
)
fedbatch_df_measurements_only['c_CO2_pseudo'] = pseudobatch_transform_pandas(
    df=fedbatch_df_measurements_only,
    measured_concentration_colnames=['preprocessed_CO2'],
    reactor_volume_colname='v_Volume',
    accumulated_feed_colname='v_Feed_accum',
    concentration_in_feed=[0],
    sample_volume_colname='sample_volume',
)

Now, we can estimate the yield coefficients from the raw and the transformed data.

In [35]:
res_yxco2_noncorrected = fit_ols_model(formula_like = "m_CO2_gas ~ m_Biomass", data= fedbatch_df_measurements_only)
res_yxco2_corrected = fit_ols_model(formula_like = "c_CO2_pseudo ~ c_Biomass_pseudo", data= fedbatch_df_measurements_only)

print(f"Fitted Yxco2 from raw data: {res_yxco2_noncorrected.params[1].round(5)}")
print(f"Fitted Yxco2 from pseudo batch transformed data: {res_yxco2_corrected.params[1].round(5)}")
print(f"True Yxco2: {fedbatch_df.Yxco2.iloc[0].round(5)}")

Fitted Yxco2 from raw data: 0.08193
Fitted Yxco2 from pseudo batch transformed data: 0.04519
True Yxco2: 0.04519


Again we are able to calculate the correct yield using the pseudo batch transformed data.

## Creating overview table
In this section we simply collects all the above results in a table.

In [36]:
overview_table_raw = pd.DataFrame.from_dict({
        "Yxs": np.abs([res_yxs_noncorrected.params[1], res_yxs_corrected.params[1], fedbatch_df.Yxs.iloc[0]]),
        "Yxp": [res_yxp_noncorrected.params[1], res_yxp_corrected.params[1], fedbatch_df.Yxp.iloc[0]],
        "Yxco2": [res_yxco2_noncorrected.params[1], res_yxco2_corrected.params[1], fedbatch_df.Yxco2.iloc[0]],
        "mu": [res_mu_hat_noncorrected.params[1], res_mu_hat_corrected.params[1], fedbatch_df.mu_true.iloc[0]],
    }, 
    columns=[
        "Non-corrected", "Corrected", "True"
    ],
    orient="index",
)

In [37]:
def relative_error(true_value, predicted_value):
    return (true_value - predicted_value) / true_value

def combine_value_and_error(value: float, error: float)-> str:
    return f"{value:.2f} ({error:.2f})"

def prepare_output_strings(true_value: float, predicted_value: float)-> str:
    error = relative_error(true_value, predicted_value)
    error_in_percent = error * 100
    return [combine_value_and_error(value=v, error=e) for v, e in zip(predicted_value, error)]


In [38]:
overview_table_clean = (
    overview_table_raw
    .assign(rel_error_noncorrected=prepare_output_strings(overview_table_raw["True"], overview_table_raw["Non-corrected"]))
    .assign(rel_error_corrected=prepare_output_strings(overview_table_raw["True"], overview_table_raw["Corrected"]))
    .round(2)
    .rename(columns={
        "rel_error_noncorrected": "Non-corrected (rel. error %)",
        "rel_error_corrected": "Corrected (rel. error %)"
    })
    .drop(columns=["Non-corrected", "Corrected"])
)
overview_table_clean

,True,Non-corrected (rel. error %),Corrected (rel. error %)
Yxs,1.85,3.36 (-0.82),1.85 (-0.00)
Yxp,0.82,0.84 (-0.02),0.82 (0.00)
Yxco2,0.05,0.08 (-0.81),0.05 (-0.00)
mu,0.10,0.05 (0.45),0.10 (-0.00)


## Simulation parameters table
In the following section, we will fetch the parameters used for the simulation. These are stored in the dataframe with the simulated data.

In [39]:
# import the dataset from the article/data folder, this makes sure if the simulations are rerun the new data is used
path_standard_fb = pathlib.Path('../data/standard_fed-batch_process.csv')
standard_fb_df = _prepare_simulated_dataset(path_standard_fb)

In [40]:
simulation_parameters = standard_fb_df[['Kc_s', 'mu_max', 'Yxs', 'Yxp', 'Yxco2', 'F0', 'mu0', 's_f']].iloc[0]
simulation_parameters['sample_volume'] = standard_fb_df['sample_volume'].unique()[1]

In [41]:
parameter_description = {
    'Kc_s': 'The Ks value for the monod equation',
    'mu_max': 'The maximum specific growth rate',
    'Yxs': 'The yield of substrate on biomass',
    'Yxp': 'The yield of product on biomass',
    'Yxco2': 'The yield of CO2 on biomass',
    'F0': 'The initial feed rate',
    'mu0': 'The target specific growth rate used to calculate the feed rate',
    's_f': 'The substrate concentration in the feed',
    'sample_volume': 'The sample volume'
}

parameter_units = {
    'Kc_s': 'g/L',
    'mu_max': '1/h',
    'Yxs': 'gSubstrate / gBiomass',
    'Yxp': 'gProduct / gBiomass',
    'Yxco2': 'gCO2 / gBiomass',
    'F0': 'µL/h',
    'mu0': '1/h',
    's_f': 'g/L',
    'sample_volume': 'µL'
}

In [42]:
simulation_parameters_df = (simulation_parameters
    .reset_index()
    .rename(columns={'index': 'Parameter symbol', 0: 'Value'})
)
simulation_parameters_df['Unit'] = simulation_parameters_df['Parameter symbol'].map(parameter_units)
simulation_parameters_df['Parameter description'] = simulation_parameters_df['Parameter symbol'].map(parameter_description)

In [43]:
simulation_parameters_df.style.format("{:.3f}", subset=['Value']).hide(level=0)

Parameter symbol,Value,Unit,Parameter description
Kc_s,0.150,g/L,The Ks value for the monod equation
mu_max,0.300,1/h,The maximum specific growth rate
Yxs,1.850,gSubstrate / gBiomass,The yield of substrate on biomass
Yxp,0.822,gProduct / gBiomass,The yield of product on biomass
Yxco2,0.045,gCO2 / gBiomass,The yield of CO2 on biomass
F0,0.063,µL/h,The initial feed rate
mu0,0.100,1/h,The target specific growth rate used to calculate the feed rate
s_f,100.000,g/L,The substrate concentration in the feed
sample_volume,170.000,µL,The sample volume
